Implementation of Layer-wise Relevance Propagation (LRP) for sentiment analysis for English Language. The package used to implement LRP is innvestigate. The dataset used is Large Movie Review Dataset (Imdb reviews). The distribution of positive and negative reviews in training and test dataset is also equal (12.5K pos and 12.5 neg reviews in each training and test dataset). The model in use is feed forwards neural network with three dense hidden layers using Keras. The vectorization technique used for text is GloVe word embedding.

Dataset location: https://ai.stanford.edu/~amaas/data/sentiment/ and extract the folder in the current directly.

LRP paper: http://iphome.hhi.de/samek/pdf/MonXAI19.pdf

GloVe word embedding: https://nlp.stanford.edu/projects/glove/

Implementation reference: https://github.com/albermax/innvestigate/blob/master/examples/notebooks/sentiment_analysis.ipynb

In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import os

import pandas as pd
import numpy as np
import re

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.core import Dense

import innvestigate

Using TensorFlow backend.


<h4>Data import</h4>

In [2]:
def parse_folder(name):
    data = []
    for verdict in ('neg', 'pos'):
        for file in glob(os.path.join(name, verdict, '*.txt')):
            data.append({
                'text': open(file, encoding='utf8').read(),
                'verdict': verdict == 'pos'
            })
    return pd.DataFrame(data)

df_train = parse_folder('../aclImdb/train/')
df_test = parse_folder('../aclImdb/test/')

df = pd.concat([df_train, df_test])
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
df = df.sample(frac=1)

In [3]:
df.head()

,text,verdict
16064,"Let me be up-front, I like pulp. However it is...",True
36542,Having enjoyed Koyaanisqatsi and Powaqatsi I w...,False
48042,"The beautiful, charming, supremely versatile a...",True
3993,To me this just comes off as a soap opera. I g...,False
38507,This is truly a funny movie. His dance scene d...,True


<h4>Basic text preprocessing</h4>

In [4]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

X_text = []
sentences = list(df['text'])
for sen in sentences:
    X_text.append(preprocess_text(sen))
    
y = df['verdict']
y = np.array(list(map(lambda x: 0 if x==True else 1, y)))

<h4>Text tokenizing & padding</h4>

In [5]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_text)
X_tokenized = tokenizer.texts_to_sequences(X_text)
vocab_size = len(tokenizer.word_index) + 1
maxlen = 200
X_padded_tokens = pad_sequences(X_tokenized, padding='post', truncating='post',maxlen=maxlen)

<h4>Word embedding</h4>

In [6]:
embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

<h4>Embedded dataset preparation</h4>

In [7]:
zero_embedding = np.zeros((100,))
embedded_X_train = np.zeros((len(X_padded_tokens), 200, 100))
for i, row in enumerate(X_padded_tokens):
    for j, token_number in enumerate(row):
        try:
            token_word = tokenizer.index_word[token_number]
            token_embedding = embeddings_dictionary[token_word]
            embedded_X_train[i,j,:] = token_embedding
        except:
            embedded_X_train[i,j,:] = zero_embedding

<h4>Keras 3 hidden layer model</h4>

In [8]:
model = Sequential()

flatten_layer = Flatten()
model.add(flatten_layer)
dense_layer_one = Dense(256, activation='relu')
model.add(dense_layer_one)
dense_layer_two = Dense(256, activation='relu')
model.add(dense_layer_two)
dense_layer_three = Dense(256, activation='relu')
model.add(dense_layer_three)
output_layer = Dense(1, activation='sigmoid')
model.add(output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


<h4>Model training</h4>

In [9]:
model.fit(embedded_X_train, y , epochs=6, batch_size=256)

Instructions for updating:
Use tf.cast instead.
Epoch 1/6
50000/50000 [==============================] - 66s 1ms/step - loss: 0.6154 - acc: 0.6760
Epoch 2/6
50000/50000 [==============================] - 64s 1ms/step - loss: 0.4698 - acc: 0.7753
Epoch 3/6
50000/50000 [==============================] - 84s 2ms/step - loss: 0.4011 - acc: 0.8163
Epoch 4/6
50000/50000 [==============================] - 60s 1ms/step - loss: 0.3437 - acc: 0.8441
Epoch 5/6
50000/50000 [==============================] - 61s 1ms/step - loss: 0.2665 - acc: 0.8808
Epoch 6/6
50000/50000 [==============================] - 61s 1ms/step - loss: 0.2010 - acc: 0.9135


<h4>5 random cases for XAI</h4>

In [10]:
random_cases = list(np.random.random_integers(0,49999,size=[5,]))
random_cases

[24338, 15011, 22339, 12563, 19677]

<h4>Initiating LRP instance</h4>

In [11]:
innvestigate_method = 'lrp.z'
analyzer = innvestigate.create_analyzer(innvestigate_method, model)

<h4>XAI preparation</h4>

In [12]:
def find_top(case_no):
    case = embedded_X_train[case_no].reshape(1,200,100)
    
    if y[case_no] == 0:
        cla = 'Positive review'
    else:
        cla = 'Negative review'
    print("True class:", cla)
    
    pred = model.predict(case)
    if pred < 0.5:
        cla = 'Positive review'
    else:
        cla = 'Negative review'
    print("Predicted class:", cla, '(',pred[0][0],')','[0=True, 1=False]')
    
    scores = np.sum(np.squeeze(analyzer.analyze(case)), axis=1)
    
    print("\nTop 10 - Positive Contribute")
    top_set = []
    for i, tk in enumerate(X_padded_tokens[case_no][scores.argsort()[-5:][::-1]]):
        try:
            top_set.append((tokenizer.index_word[tk], scores[scores.argsort()[-5:][::-1]][i]))
        except:
            1
    print(top_set)
    
    print("\nTop 10 - Negative Contribute")
    bottom_set = []
    for i, tk in enumerate(X_padded_tokens[case_no][np.flip(scores.argsort()[:5][::-1])]):
        try:
            bottom_set.append((tokenizer.index_word[tk], scores[scores.argsort()[:5][::-1]][i]))
        except:
            1
    print(bottom_set)
    
    print("\nText:\n", X_text[case_no])
    print("\n----------------------------------------------")

<h4>XAI</h4>

In [14]:
for i in random_cases:
    find_top(i)

True class: Negative review
Predicted class: Negative review ( 0.9515501 ) [0=True, 1=False]

Top 10 - Positive Contribute
[('ed', 0.23428376), ('running', 0.19387451), ('believe', 0.1913349), ('not', 0.18128063), ('plot', 0.16896152)]

Top 10 - Negative Contribute
[('him', -0.1360347), ('film', -0.14448667), ('who', -0.16167434), ('introduces', -0.17170446), ('ray', -0.19200623)]

Text:
 I was ed when couldn see this one when it was screening at the Philly Film Fest last year so when saw that it was going to be on cable tonight put it on remind as soon as could So was it worth the wait Well let backtrack tad as have yet to give you the plot Sean Crawley is young man who doesn know what his path in life is Enter Duke George Wendt who introduces him to his boss Ray Danny Baldwin One night Ray totally hammered asks Sean to off the guy that they had Sean following around And it goes on from there Which leads me back to the question posed Was it worth the wait Yes and no the buildup was pr

<h4>Insights</h4>

The LRP method is suitable for making a trust in the model individual prediction. But it lacks method to build trust in the model. As can be seen from LRP results: 
1. Despite being model accuracy being 91%, it seems the current model lacks in learning the important words for classifying the text.
2. It highlights the need to remove stop words